In [127]:
from bs4 import BeautifulSoup
import urllib.request
import re
import json
import pandas as pd
from datetime import date
from tqdm.auto import tqdm
import ipywidgets

In [128]:
today = str(date.today())

In [129]:
dummy_url = "http://memory.pvost.org/pages/dic.html"
page = urllib.request.urlopen(dummy_url)

In [130]:
#ищем все линки
soup = BeautifulSoup(page)
links = []
for item in soup("li", {"class": "main"}):
    for link in item.find_all("a"):
        if "href" in link.attrs:
            links.append(link["href"])

In [131]:
#подготовленные имена для определения гендера
ambig_fem = ["Любовь", "Рашель", "Эсфирь", "Элен", "Нуне", "Исоко", "Нинель", "Мариам"]
ambig_male = ["Абдулла", "Илья", "Дзига", "Никита", "Махмуд-ходжа", "Шевки", "Бекги", "Текки", "Шалва", "Вали", "Мамия", "Эльбек-Доржи", "Абони", "Грачия", "Рухулла", "Мяти", "Цви", "Чайжи-Дава", "Хаджи", "Габдел-Бари", "Ахмет-Заки", "Бидия", "Гази-Али", "Даши", "Хади", "Ханафи", "Баруди", "Али", "Хужа", "Шота", "Хамди", "Абибулла", "Вели", "Ягья", "Мухаметша", "Ахмедхади", "Амди", "Габдулла", "Ушана", "Эрдени", "Осман Нури"]

In [143]:
#собираем словарь с данными
biographies = []
for link in tqdm(links): 
    page = "http://memory.pvost.org/pages/" + link
    try:
        page = urllib.request.urlopen(page)
    except Exception:
        continue
    soup_bio = BeautifulSoup(page)
    east_bio = {}
    info = soup_bio.find("p", {"class": "zag2"})
    biography = soup_bio.find("p", attrs={'class': None})
    biography = re.sub(r"\n\s{2,30}", "", biography.text)
    names = info.text
    names = re.sub(r'\([^()]*\)', "", names)
    names = re.sub(r'\([^()]*\)', "", names)
    try:
        dates = re.search(r"((\d{4}\??|\?)-?(\d{4}\??|\?)?)", info.text).group()
    except AttributeError:
        continue
    try:
        surname = re.search(r"[А-ЯЁ]+-?[А-ЯЁ]+", names).group()
        east_bio["surname"] = surname.title()
    except AttributeError:
        east_bio["surname"] = ""
    try: 
        birth = re.search(r"(\d{4}\??|\?)", dates).group()
    except AttributeError:
        continue
    try:
        firstname = re.search(r"([А-ЯЁ]{1}[а-яё]+-?[А-Яа-яЁё]{0,10})|([А-ЯЁ]\.)", names).group()
        east_bio["firstname"] = firstname
    except AttributeError:
        firstname = ""
        east_bio["firstname"] = firstname
    #try:
    #    firstname = re.search(r"([А-Я]{1}[а-я]+-?[А-Яа-я]+)", names).group()
    #except AttributeError:
    #    continue
    try:
        patronym = re.search(r"([А-ЯЁ]{1}[а-яё]+-?[А-Яа-яЁё]+)|([А-ЯЁ]\.)", names.split(firstname+" ")[1]).group()
        east_bio["patronym"] = patronym
        east_bio["fullname"] = surname.title()+" "+firstname+" "+patronym
    except AttributeError:
        patronym = ""
        east_bio["patronym"] = patronym
        east_bio["fullname"] = surname.title()+" "+firstname
    except IndexError:
        patronym = ""
        east_bio["patronym"] = patronym
        east_bio["fullname"] = surname.title()+" "+firstname
    if re.search("(а|я|и)$", firstname) and firstname not in ambig_male:
        east_bio["gender"] = "женщина"
    elif firstname in ambig_fem:
        east_bio["gender"] = "женщина"
    else:
        east_bio["gender"] = "мужчина"
    #try:
    #    east_bio["fullname"] = surname.title()+" "+firstname+" "+patronym
    #except NameError:
    #    east_bio["fullname"] = surname.title()+" "+firstname
    try:
        east_bio["birthdate"] = birth
    except AttributeError:
        east_bio["birthdate"] = ""
    try:
        east_bio["birthplace"] = re.search(r"[А-ЯЁ]{0,2}[а-яё]{1,7}\.?[А-Яа-яЁё\s-]+(?=\.|,|;)", biography.split("Род. в ")[1]).group()
    except IndexError:
        east_bio["birthplace"] = ""
    except AttributeError:
        east_bio["birthplace"] = ""
    try:
        east_bio["death_date"] = re.search(r"1[8-9][0-9]{2}", dates.split(birth)[1]).group()
    except AttributeError:
        east_bio["death_date"] = ""
    except IndexError:
        east_bio["death_date"] = ""
    east_bio["original_link"] = "http://memory.pvost.org/pages/" + link
    east_bio["retrieved_at"] = today
    biographies.append(east_bio)

  0%|          | 0/771 [00:00<?, ?it/s]

In [108]:
#смотрим, что получилось в датафрейме
bio_parse = pd.DataFrame(biographies)

In [109]:
bio_parse

,surname,firstname,patronym,fullname,gender,birthdate,birthplace,death_date,original_link,retrieved_at
0,Абих,Рудольф,Петрович,Абих Рудольф Петрович,мужчина,1901,г. Баку,1940,http://memory.pvost.org/pages/abrih.html,2021-04-09
1,Абрамский,Иехезкель,,Абрамский Иехезкель,мужчина,1886,м. Дашковичи Гродненской губ,1976,http://memory.pvost.org/pages/abramskij.html,2021-04-09
2,Абрамсон,Мануил,Моисеевич,Абрамсон Мануил Моисеевич,мужчина,1898,др. свед,1938,http://memory.pvost.org/pages/abramson.html,2021-04-09
3,Абугов,Михаил,Захарович,Абугов Михаил Захарович,мужчина,1901,зам. зав,1941,http://memory.pvost.org/pages/abugov.html,2021-04-09
4,Абузов,Степан,Семенович,Абузов Степан Семенович,мужчина,1898,с. Старые Бегучи Самарской губ,1938,http://memory.pvost.org/pages/abuzov.html,2021-04-09
...,...,...,...,...,...,...,...,...,...,...
737,Чернышев,Евгений,Иванович,Чернышев Евгений Иванович,мужчина,1894,г. Симбирск,,http://memory.pvost.org/pages/ds_chernishev.html,2021-04-09
738,Шийк,Андрей,Александрович,Шийк Андрей Александрович,мужчина,1891,,1978,http://memory.pvost.org/pages/ds_shijk.html,2021-04-09
739,Элиасов,Лазарь,Ефимович,Элиасов Лазарь Ефимович,мужчина,?,,,http://memory.pvost.org/pages/ds_elisarov.html,2021-04-09
740,Янчевецкий,Дмитрий,Григорьевич,Янчевецкий Дмитрий Григорьевич,мужчина,1889,,1934,http://memory.pvost.org/pages/ds_yanchevitskij...,2021-04-09


In [49]:
names = set()
for item in bio_parse["firstname"]:
    names.add(item)

In [57]:
female = []
male = []
ambig_fem = ["Любовь", "Рашель", "Эсфирь", "Элен", "Нуне", "Исоко", "Нинель", "Мариам"]
ambig_male = ["Абдулла", "Илья", "Дзига", "Никита", "Махмуд-ходжа", "Шевки", "Бекги", "Текки", "Шалва", "Вали", "Мамия", "Эльбек-Доржи", "Абони", "Грачия", "Рухулла", "Мяти", "Цви", "Чайжи-Дава", "Хаджи", "Габдел-Бари", "Ахмет-Заки", "Бидия", "Гази-Али", "Даши", "Хади", "Ханафи", "Баруди", "Али", "Хужа", "Шота", "Хамди", "Абибулла", "Вели", "Ягья", "Мухаметша", "Ахмедхади", "Амди", "Габдулла", "Ушана", "Эрдени"] 
for name in names:
    if re.search("(а|я|и)$", name) and name not in ambig_male:
        female.append(name)
    elif name in ambig_fem:
        female.append(name)
    else:
         male.append(name)

In [73]:
male

['',
 'Мемет',
 'Цирен',
 'Всеволод',
 'Павел',
 'Махмуд-ходжа',
 'Альберт',
 'Авдышо',
 'Семен',
 'Аркадий',
 'Шевки',
 'Бекги',
 'Агван-Лобсан',
 'Артур',
 'Алексан',
 'Текки',
 'Бруно',
 'Эквтиме',
 'Марат',
 'Нурэддин',
 'Гаяз',
 'Ростислав',
 'Бекир',
 'Хан',
 'Иннокентий',
 'Вильгельм',
 'Дашипилон',
 'Илиас',
 'Лаврентий',
 'Санджар',
 'Петр',
 'Вячеслав',
 'Василий',
 'Шалва',
 'Карабек',
 'Антон',
 'Петрович',
 'Вали',
 'Милий',
 'Леон',
 'Иосиф',
 'Роберт',
 'Мануил',
 'Альфред',
 'Эммануил',
 'Матвей',
 'Халид',
 'Миноро',
 'Мозес',
 'Исаак',
 'Отто',
 'Елеазар',
 'Якуб',
 'Мамия',
 'Ашхарбек',
 'Эльбек-Доржи',
 'Алибек',
 'Базаржаб',
 'Мерген',
 'Дин-шунь',
 'Нигмат',
 'Абдул-Керим',
 'Степан',
 'Джамал',
 'Абони',
 'Венедикт',
 'Ибраим',
 'Шариф',
 'Грачия',
 'Пент',
 'Балжир',
 'Ризай',
 'Нисар',
 'Салах',
 'Рухулла',
 'Мяти',
 'Гуан-Сик',
 'Абульфаз',
 'Юсиф',
 'Вукол',
 'Ен-Дюн',
 'Арутюн',
 'Донатас',
 'Александр',
 'Наум',
 'Амплий',
 'Абдул-Азиз',
 'Фердинанд',
 'Цви

In [59]:
female_regex = re.compile(""+"|".join(female))
#regex = re.compile("^("+"|".join(item_list)+")\.")
male_regex = re.compile(""+"|".join(male))

In [62]:
gender = []
for item in bio_parse["firstname"]:
    fem_match = re.search(female_regex, str(item))
    male_match = re.search(male_regex, str(item))
    if fem_match:
        gender.append("женщина")
    elif male_match:
        gender.append("мужчина")
    else:
        gender.append("")
bio_parse["gender"] = gender

In [63]:
bio_parse

,surname,firstname,patronym,fullname,birthdate,death_date,birthplace,original_link,retrieved_at,gender
0,Абих,Рудольф,Петрович,Абих Рудольф Петрович,1901,1940,г. Баку,http://memory.pvost.org/pages/abrih.html,2021-04-09,мужчина
1,Абрамский,Иехезкель,,Абрамский Иехезкель,1886,1976,м. Дашковичи Гродненской губ,http://memory.pvost.org/pages/abramskij.html,2021-04-09,мужчина
2,Абрамсон,Мануил,Моисеевич,Абрамсон Мануил Моисеевич,1898,1938,др. свед,http://memory.pvost.org/pages/abramson.html,2021-04-09,мужчина
3,Абугов,Михаил,Захарович,Абугов Михаил Захарович,1901,1941,зам. зав,http://memory.pvost.org/pages/abugov.html,2021-04-09,мужчина
4,Абузов,Степан,Семенович,Абузов Степан Семенович,1898,1938,с. Старые Бегучи Самарской губ,http://memory.pvost.org/pages/abuzov.html,2021-04-09,мужчина
...,...,...,...,...,...,...,...,...,...,...
733,Чернышев,Евгений,Иванович,Чернышев Евгений Иванович,1894,,г. Симбирск,http://memory.pvost.org/pages/ds_chernishev.html,2021-04-09,мужчина
734,Шийк,Андрей,Александрович,Шийк Андрей Александрович,1891,1978,,http://memory.pvost.org/pages/ds_shijk.html,2021-04-09,мужчина
735,Элиасов,Лазарь,Ефимович,Элиасов Лазарь Ефимович,?,,,http://memory.pvost.org/pages/ds_elisarov.html,2021-04-09,мужчина
736,Янчевецкий,Дмитрий,Григорьевич,Янчевецкий Дмитрий Григорьевич,1889,1934,,http://memory.pvost.org/pages/ds_yanchevitskij...,2021-04-09,мужчина


In [125]:
#тестовый парсинг одной статьи
#dummy_url_again = "http://memory.pvost.org/pages/akki.html"
dummy_url_again = "http://memory.pvost.org/pages/adrianovaa.html"
page_east = urllib.request.urlopen(dummy_url_again)
soup_bio = BeautifulSoup(page_east)

In [126]:
east_bio = {}
info = soup_bio.find("p", {"class": "zag2"})
biography = soup_bio.find("p", attrs={'class': None})
biography = re.sub(r"\n\s{2,30}", "", biography.text)
names = info.text
names = re.sub(r'\([^()]*\)', "", names)
names = re.sub(r'\([^()]*\)', "", names)
#names = re.sub(r'\(.*\)(.*\))?', "", names)
dates = re.search(r"((\d{4}\??|\?)-?(\d{4}\??|\?)?)", info.text).group()
surname = re.search(r"[А-ЯЁ]+-?[А-ЯЁ]+", names).group()
east_bio["surname"] = surname.title()
firstname = re.search(, names).group()
east_bio["firstname"] = firstname.title()
birth = re.search(r"(\d{4}\??|\?)", dates).group()

try:
    patronym = re.search(r"([А-ЯЁ]{1}[а-яё]+-?[А-Яа-яЁё]+)", names.split(firstname+" ")[1]).group()
    east_bio["patronym"] = patronym
except AttributeError:
    east_bio["patronym"] = ""
#    east_bio["birth_year"] = re.search(r"1[8-9][0-9]{2}", info.text.split("-")[0]).group()

try:
    east_bio["fullname"] = surname.title()+" "+firstname+" "+patronym
except NameError:
    east_bio["fullname"] = surname.title()+" "+firstname
if re.search("(а|я|и)$", firstname) and firstname not in ambig_male:
    east_bio["gender"] = "женщина"
elif firstname in ambig_fem:
    east_bio["gender"] = "женщина"
else:
    east_bio["gender"] = "мужчина"
try:
    east_bio["birthdate"] = birth
except AttributeError:
    east_bio["birthdate"] = ""
try:
    east_bio["death_date"] = re.search(r"1[8-9][0-9]{2}", dates.split(birth)[1]).group()
except IndexError:
    east_bio["death_date"] = ""
except AttributeError:
    east_bio["death_date"] = ""
try: 
    east_bio["birthplace"] = re.search(r"[а-яё]{1,3}\.{1}[А-Яа-яЁё\s]+(?=\.|,)", biography.split("Род. в ")[1]).group()
except IndexError:
    east_bio["birthplace"] = ""
print(east_bio)
print(names)

{'surname': 'Адрианов', 'firstname': 'Александр', 'patronym': 'Александрович', 'fullname': 'Адрианов Александр Александрович', 'gender': 'мужчина', 'birthdate': '1882', 'death_date': '', 'birthplace': 'г. Томск'}
АДРИАНОВ 
                      Александр 
                      Александрович 
                      


In [142]:
with open('vostokovedy4_base.json', 'w', encoding='utf-8') as f:
    json.dump(biographies, f, ensure_ascii=False, indent=4)

In [144]:
biographies

[{'surname': 'Абих',
  'firstname': 'Рудольф',
  'patronym': 'Петрович',
  'fullname': 'Абих Рудольф Петрович',
  'gender': 'мужчина',
  'birthdate': '1901',
  'birthplace': 'г. Баку',
  'death_date': '1940',
  'original_link': 'http://memory.pvost.org/pages/abrih.html',
  'retrieved_at': '2021-04-14'},
 {'surname': 'Абрамский',
  'firstname': 'Иехезкель',
  'patronym': '',
  'fullname': 'Абрамский Иехезкель',
  'gender': 'мужчина',
  'birthdate': '1886',
  'birthplace': 'м. Дашковичи Гродненской губ',
  'death_date': '1976',
  'original_link': 'http://memory.pvost.org/pages/abramskij.html',
  'retrieved_at': '2021-04-14'},
 {'surname': 'Абрамсон',
  'firstname': 'Мануил',
  'patronym': 'Моисеевич',
  'fullname': 'Абрамсон Мануил Моисеевич',
  'gender': 'мужчина',
  'birthdate': '1898',
  'birthplace': 'колонии Львово Харьковской губ',
  'death_date': '1938',
  'original_link': 'http://memory.pvost.org/pages/abramson.html',
  'retrieved_at': '2021-04-14'},
 {'surname': 'Абугов',
  'fir